## 17807671 Shivam Pandey
### Assignment3 CE784A
> For complete source code pl. refer to following repositories
> https://github.com/ShivamPR21/ModuleZooTorch
> https://github.com/ShivamPR21/Driver-Gaze-Zone-Classification
> Code credit @ShivamPR21

### Import relevent modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import torch.nn as nn
import torchsummary
import torch
import torch.optim as optim

from dgzc.dataset import DGZCInferenceDataset
from dgzc.classifier import ClassificationBackbone2

### Define training dataset

In [2]:
data_path = "/home/shivam/2021-22-2/ML4CE/Assignments/Assignment3/Driver-Gaze-Zone-Classification/data/gaze_dataset"
# dataset_auto_enc = DGZCAutoEncoderDataset(data_path, size=(200, 200))
# dataset_classifier = DGZCClassifierDataset(data_path, size=(200, 200))
dataset_inference = DGZCInferenceDataset(data_path, size=(200, 200))

In [3]:
inv_class_map = {0 : 'Centerstack',
                 1 : 'Forward',
                 2 : 'Left_wing_mirror',
                 3 : 'Rearview_mirror',
                 4 : 'Right_wing_mirror',
                 -1 : 'other'}

### Create the dataloader with batch size `10`

In [4]:
dataloader = DataLoader(dataset_inference, batch_size = 10, shuffle = False, num_workers=5)

### Set the device to GPU if available

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
data_len = len(dataloader)
log_n = int(data_len//3)

### Create model instance

In [7]:
model = ClassificationBackbone2(l=1.)
model.to(device)

ClassificationBackbone2(
  (encoder): Encoder(
    (conv1): Conv2DNormActivation(
      (0): Conv2d(3, 5, kernel_size=(5, 5), stride=(2, 2))
      (1): SELU()
    )
    (conv2): Conv2DNormActivation(
      (0): Conv2d(5, 10, kernel_size=(5, 5), stride=(1, 1), bias=False)
      (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SELU()
    )
    (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2DNormActivation(
      (0): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
      (1): SELU()
    )
    (conv4): Conv2DNormActivation(
      (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
      (1): SELU()
    )
    (conv5): Conv2DNormActivation(
      (0): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SELU()
    )
    (conv6): Conv2DNormActivation(
      (0): Conv2d(30, 4

### Load pretrained weight dicts

In [8]:
load_prev_auto_enc_state = False
load_prev_model_state = True
if load_prev_auto_enc_state:
    model.encoder.load_state_dict(torch.load('./encoder_state'))
elif load_prev_model_state:
    model.load_state_dict(torch.load('./simple_backbone2_model'))

### View model summary

In [9]:
torchsummary.summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 5, 98, 98]             380
              SELU-2            [-1, 5, 98, 98]               0
            Conv2d-3           [-1, 10, 94, 94]           1,250
       BatchNorm2d-4           [-1, 10, 94, 94]              20
              SELU-5           [-1, 10, 94, 94]               0
         MaxPool2d-6           [-1, 10, 47, 47]               0
            Conv2d-7           [-1, 20, 45, 45]           1,820
              SELU-8           [-1, 20, 45, 45]               0
            Conv2d-9           [-1, 20, 43, 43]           3,620
             SELU-10           [-1, 20, 43, 43]               0
           Conv2d-11           [-1, 30, 41, 41]           5,400
      BatchNorm2d-12           [-1, 30, 41, 41]              60
             SELU-13           [-1, 30, 41, 41]               0
           Conv2d-14           [-1, 40,

### Set model as non-trainable

In [10]:
model.eval()

ClassificationBackbone2(
  (encoder): Encoder(
    (conv1): Conv2DNormActivation(
      (0): Conv2d(3, 5, kernel_size=(5, 5), stride=(2, 2))
      (1): SELU()
    )
    (conv2): Conv2DNormActivation(
      (0): Conv2d(5, 10, kernel_size=(5, 5), stride=(1, 1), bias=False)
      (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SELU()
    )
    (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2DNormActivation(
      (0): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
      (1): SELU()
    )
    (conv4): Conv2DNormActivation(
      (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
      (1): SELU()
    )
    (conv5): Conv2DNormActivation(
      (0): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SELU()
    )
    (conv6): Conv2DNormActivation(
      (0): Conv2d(30, 4

### Find the labels, from model

In [11]:
labels = []

In [12]:
for i, (data, target) in enumerate(dataloader):
    data = data.to(device) # Move data to target device

    cls = model(data)

    predictions = cls.max(dim=1)[1]
    labels += list(predictions.detach().cpu().numpy())

### Convert index labels to names

In [19]:
str_labels = [inv_class_map[x] for x in labels]

In [20]:
f_names = [x.split('/')[-1] for x in dataset_inference.images_path]

In [21]:
int_f_names = [int(x.split('.')[0]) for x in f_names]

### Export predictions to `.csv` file

In [22]:
idx = np.argsort(int_f_names)

In [23]:
data = np.array([f_names, str_labels]).T
data = data[idx]
result_df = pd.DataFrame(data, columns=['filename', 'class'])

In [24]:
result_df.to_csv('./result5.csv', index=False)